In [37]:
!pip install anytree

     |████████████████████████████████| 41 kB 163 kB/s eta 0:00:01


In [34]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import qgrid
import json
from anytree import Node, RenderTree

In [2]:
pd.options.display.max_rows = 1000 
pd.options.display.max_columns = 1000 
pd.options.display.max_colwidth = 1000
 
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>")) 

In [26]:
# https://raw.githubusercontent.com/hunters-forge/openhunt/master/openhunt/logparser.py

# class winlogbeat(object):
	
# 	# Function to read a json file
# 	def get_mordorDF(self, path, spark=False):
# 		print("[+] Reading Mordor file..")
# 		if (spark):
# 			mordorDF = spark.read.json(path)
# 		else:
# 			mordorDF= pd.read_json(path, lines = True)
# 		return mordorDF
	
# 	# Function to parse winlogbeat data up to version 6
# 	def winlogbeat_6(self, mordorDF, df_type):
# 		print("[+] Processing Data from Winlogbeat version 6..")
# 		if df_type == 'Pandas':
# 			# Extract event_data nested fields
# 			event_data_field = mordorDF['event_data'].apply(pd.Series)
# 			mordorDF = mordorDF.drop('event_data', axis = 1)
# 			mordorDF_flat = pd.concat([mordorDF, event_data_field], axis = 1)
# 			mordorDF= mordorDF_flat\
# 				.dropna(axis = 1,how = 'all')\
# 				.rename(columns={'log_name':'channel','record_number':'record_id','source_name':'provider_name'})
# 		elif df_type == 'Spark':
# 			mordorDF = mordorDF.select('event_data.*','source_name','log_name','record_number','event_id','computer_name','@timestamp','message')
# 			mordorDF = mordorDF\
# 				.withColumnRenamed("log_name", "channel")\
#     			.withColumnRenamed("source_name", "provider_name")\
#     			.withColumnRenamed("record_number","record_id")
# 		else:
# 			exit
# 		return mordorDF
	
# 	# Function to parse winlogbeat data since version 7
# 	def winlogbeat_7(self, mordorDF, df_type):
# 		print("[+] Processing Data from Winlogbeat version 7..")
# 		if df_type == 'Pandas':
# 			winlog_field = mordorDF['winlog'].apply(pd.Series)
# 			event_data_field = winlog_field['event_data'].apply(pd.Series)
# 			event_data_field = self.clean_fields(event_data_field)
# 			mordorDF_flat = pd.concat([mordorDF, winlog_field, event_data_field], axis = 1)
# 			mordorDF= df.dropna(axis = 1,how = 'all').drop(['winlog','event_data','process'], axis = 1)
# 			mordorDF['level'] = mordorDF['log'].apply(lambda x : x.get('level'))
# 		elif df_type == 'Spark':
# 			mordorDF = mordorDF.select('winlog.event_data.*','winlog.channel','winlog.provider_name','winlog.record_id','winlog.event_id','winlog.computer_name','@timestamp','message')
# 			mordorDF = mordorDF\
# 				.withColumnRenamed("winlog.channel", "channel")\
#     			.withColumnRenamed("winlog.provider_name", "provider_name")\
#     			.withColumnRenamed("winlog.record_id","record_id")\
# 				.withColumnRenamed("winlog.event_id","event_id")\
# 				.withColumnRenamed("winlog.computer_name","computer_name")
# 		else:
# 			exit
# 		return mordorDF
	
# 	# Function to parse winlogbeat data (all versions)
# 	def extract_nested_fields(self, path, spark=False):
# 		if (spark):
# 			print("[+] Processing a Spark DataFrame..")
# 			mordorDF= self.get_mordorDF(path, spark)
# 			mordorDF = mordorDF.withColumn('version', mordorDF["@metadata.version"].substr(1,1))
# 			mordorDF = mordorDF.withColumn('beat_type', mordorDF["@metadata.beat"])
# 			# Verify what verion of Winlogbeat was used to ship the data
# 			if (len(mordorDF.filter((mordorDF.beat_type == 'winlogbeat') & (mordorDF.version <= 6)).limit(1).take(1)) > 0 ):
# 				mordorDF_return = self.winlogbeat_6(mordorDF.filter((mordorDF.beat_type == 'winlogbeat') & (mordorDF.version <= 6)), 'Spark')
# 			elif (len(mordorDF.filter((mordorDF.beat_type == 'winlogbeat') & (mordorDF.version >= 7)).limit(1).take(1)) > 0 ):
# 				mordorDF_return = self.winlogbeat_7(mordorDF.filter((mordorDF.beat_type == 'winlogbeat') & (mordorDF.version >= 7)),'Spark')
# 			else:
# 				exit
# 		else:
# 			print("[+] Processing Pandas DataFrame..")
# 			mordorDF= self.get_mordorDF(path)
# 			mordorDF['version'] = mordorDF['@metadata'].apply(lambda x : x.get('version'))
# 			mordorDF['version'] = mordorDF['version'].astype(str).str[0]
# 			mordorDF['beat_type'] = mordorDF['@metadata'].apply(lambda x : x.get('beat'))
# 			# Initialize Empty Dataframe
# 			mordorDF_return = pd.DataFrame()
# 			# Verify what verion of Winlogbeat was used to ship the data
# 			if ((mordorDF['beat_type'] == 'winlogbeat') & (mordorDF['version'] <= '6')).any():
# 				version_6_df = self.winlogbeat_6(mordorDF[(mordorDF['beat_type'] == 'winlogbeat') & (mordorDF['version'] <= '6')], 'Pandas')
# 				mordorDF_return = mordorDF_return.append(version_6_df, sort = False)		
# 			if ((mordorDF['beat_type'] == 'winlogbeat') & (mordorDF['version'] >= '7')).any():
# 				version_7_df = self.winlogbeat_7(mordorDF[(mordorDF['beat_type'] == 'winlogbeat') & (mordorDF['version'] >= '7')], 'Pandas')
# 				mordorDF_return = mordorDF_return.append(version_7_df, sort = False)			
# 			if (mordorDF['beat_type'] != 'winlogbeat').any():
# 				not_winlogbeat = mordorDF[mordorDF['beat_type'] != 'winlogbeat']
# 				mordorDF_return = mordorDF_return.append(not_winlogbeat, sort = False)		
# 			mordorDF_return = mordorDF_return.dropna(axis = 0,how = 'all').reset_index(drop = True)

# 		print("[+] DataFrame Returned !")
# 		return mordorDF_return

In [ ]:
# mordor = winlogbeat()
# mordor.extract_nested_fields(data)

In [75]:
data = '/home/rich/Downloads/empire_apt3/empire_apt3_2019-05-14223117.json'

mordor_df = pd.read_json(data, lines=True)

sysmon_df = mordor_df[mordor_df['source_name'] == 'Microsoft-Windows-Sysmon']
sysmon_df = sysmon_df.dropna(axis='columns', how='all')

# Break out all nested json fields into their own series, which we will rejoin via concat
event_data_field = json_normalize(sysmon_df['event_data'])
user_data_field = json_normalize(sysmon_df['user'])

# Drop fields we don't want, and ones that are parsed.
sysmon_df = sysmon_df.drop(['event_data', '@metadata', 'message', 'host', 'beat', 'user'], axis='columns')
sysmon_df_flat = pd.concat([sysmon_df, event_data_field, user_data_field, provider_guid_field], axis='columns')

# Rename fields
sysmon_df = sysmon_df_flat.dropna(axis='columns',how = 'all').rename(columns={'log_name':'channel','record_number':'record_id','source_name':'provider_name', 'name':'username', 'identifier':'sid'})

# Strip curly braces from fields. There's probably a better way to do this.
sysmon_df['ProcessGuid'] = sysmon_df['ProcessGuid'].str.replace('}','').str.replace('{', '')
sysmon_df['ParentProcessGuid'] = sysmon_df['ParentProcessGuid'].str.replace('}','').str.replace('{', '')
sysmon_df['SourceProcessGUID'] = sysmon_df['SourceProcessGUID'].str.replace('}','').str.replace('{', '')
sysmon_df['TargetProcessGUID'] = sysmon_df['TargetProcessGUID'].str.replace('}','').str.replace('{', '')
sysmon_df['provider_guid'] = sysmon_df['provider_guid'].str.replace('}','').str.replace('{','')

In [81]:
# THE DATA IS GETTING MESSED UP HERE SOMEHOW. LINE 983 IS ALL NULL IN PROCESS. IT SEEMS TO BE A NETWORK CONNECTION THAT GOT JOINED.
print(mordor_df.iloc[983])

@timestamp                                                                                                                                                                                                                                                                                                                                                                                                                                                                             2019-05-14T22:31:38.441Z
@metadata                                                                                                                                                                                                                                                                                                                                                                                                                      {'beat': 'winlogbeat', 'type': 'doc', 'version': '6.7.0', 'topic': 'winlogbeat'}
log_name

In [79]:
event_data_field[~event_data_field['CommandLine'].isna()].head()

,TargetProcessId,SourceThreadId,SourceImage,GrantedAccess,CallTrace,UtcTime,SourceProcessGUID,TargetProcessGUID,SourceProcessId,TargetImage,SourceHostname,SourceIsIpv6,Protocol,ProcessGuid,SourceIp,SourcePortName,DestinationIp,DestinationPort,ProcessId,DestinationIsIpv6,SourcePort,Image,User,Initiated,DestinationHostname,PipeName,Company,Signed,Hashes,SignatureStatus,Product,ImageLoaded,FileVersion,Description,Signature,TargetObject,EventType,DestinationPortName,Details,Device,CommandLine,TerminalSessionId,CurrentDirectory,ParentProcessId,ParentCommandLine,IntegrityLevel,ParentProcessGuid,LogonGuid,LogonId,ParentImage,TargetFilename,CreationUtcTime,PreviousCreationUtcTime,Hash
241,NaN,NaN,NaN,NaN,NaN,2019-05-14 22:31:27.319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{03ba39f5-41bf-5cdb-0000-00104c568400},NaN,NaN,NaN,NaN,3336,NaN,NaN,C:\Windows\System32\taskhostw.exe,SHIRE\nmartha,NaN,NaN,NaN,Microsoft Corporation,NaN,"SHA1=121E6C3CF7AF4CA30DB43440F71C1510EC3EA261,MD5=F225F37717C8D714B56CF21C7C1E9C34,SHA256=E272D0E33299BCA60479683EAE0D9B5D880293C01B337D5759FEE812EA631A39,IMPHASH=9839C7FD9649496B162F72128209528A",NaN,Microsoft® Windows® Operating System,NaN,10.0.17763.1 (WinBuild.160101.0800),Host Process for Windows Tasks,NaN,NaN,NaN,NaN,NaN,NaN,taskhostw.exe KEYROAMING,1,C:\Windows\system32\,1048,C:\Windows\system32\svchost.exe -k netsvcs -p -s Schedule,Medium,{03ba39f5-ea64-5ccb-0000-0010c91c0100},{03ba39f5-e67a-5cda-0000-0020c40c0c00},0xc0cc4,C:\Windows\System32\svchost.exe,NaN,NaN,NaN,NaN
315,NaN,NaN,NaN,NaN,NaN,2019-05-14 22:31:27.463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{03ba39f5-41bf-5cdb-0000-0010785d8400},NaN,NaN,NaN,NaN,1128,NaN,NaN,C:\Windows\System32\backgroundTaskHost.exe,SHIRE\nmartha,NaN,NaN,NaN,Microsoft Corporation,NaN,"SHA1=339E4E69D2120B97CE34B9A8D3597FF8E0A73561,MD5=50D5FD1290D94D46ACCA0585311E74D5,SHA256=B8E176FE76A1454A00C4AF0F8BF8870650D9C33D3E333239A59445C5B35C9A37,IMPHASH=D2ACF1CBC4A6DB14A34C687B9362D66B",NaN,Microsoft® Windows® Operating System,NaN,10.0.17763.1 (WinBuild.160101.0800),Background Task Host,NaN,NaN,NaN,NaN,NaN,NaN,"""C:\Windows\system32\backgroundTaskHost.exe"" -ServerName:CortanaUI.AppXy7vb4pc2dr3kc93kfc509b1d0arkfb2x.mca",1,C:\Windows\SystemApps\Microsoft.Windows.Cortana_cw5n1h2txyewy\,748,C:\Windows\system32\svchost.exe -k DcomLaunch -p,AppContainer,{03ba39f5-ea62-5ccb-0000-001096a70000},{03ba39f5-e67a-5cda-0000-0020c40c0c00},0xc0cc4,C:\Windows\System32\svchost.exe,NaN,NaN,NaN,NaN
364,NaN,NaN,NaN,NaN,NaN,2019-05-14 22:31:27.680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{03ba39f5-41bf-5cdb-0000-0010416a8400},NaN,NaN,NaN,NaN,2396,NaN,NaN,C:\Windows\System32\backgroundTaskHost.exe,SHIRE\nmartha,NaN,NaN,NaN,Microsoft Corporation,NaN,"SHA1=339E4E69D2120B97CE34B9A8D3597FF8E0A73561,MD5=50D5FD1290D94D46ACCA0585311E74D5,SHA256=B8E176FE76A1454A00C4AF0F8BF8870650D9C33D3E333239A59445C5B35C9A37,IMPHASH=D2ACF1CBC4A6DB14A34C687B9362D66B",NaN,Microsoft® Windows® Operating System,NaN,10.0.17763.1 (WinBuild.160101.0800),Background Task Host,NaN,NaN,NaN,NaN,NaN,NaN,"""C:\Windows\system32\backgroundTaskHost.exe"" -ServerName:CortanaUI.AppXy7vb4pc2dr3kc93kfc509b1d0arkfb2x.mca",1,C:\Windows\SystemApps\Microsoft.Windows.Cortana_cw5n1h2txyewy\,748,C:\Windows\system32\svchost.exe -k DcomLaunch -p,AppContainer,{03ba39f5-ea62-5ccb-0000-001096a70000},{03ba39f5-e67a-5cda-0000-0020c40c0c00},0xc0cc4,C:\Windows\System32\svchost.exe,NaN,NaN,NaN,NaN
760,NaN,NaN,NaN,NaN,NaN,2019-05-14 22:31:32.624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{03ba39f5-41c4-5cdb-0000-0010d9af8400},NaN,NaN,NaN,NaN,5900,NaN,NaN,C:\Windows\System32\svchost.exe,NT AUTHORITY\SYSTEM,NaN,NaN,NaN,Microsoft Corporation,NaN,"SHA1=A1385CE20AD79F55DF235EFFD9780C31442AA234,MD5=8A0A29438052FAED8A2532DA50455756,SHA256=7FD065BAC18C5278777AE44908101CDFED72D26FA741367F0AD4D02020787AB6,IMPHASH=247B9220E5D9B720A82B2C8B5069AD69",NaN,Microsoft® Windows® Operating System,NaN,10.0.17763.1 (WinBuild.160101.0800),Host Process for Windows Services,NaN,NaN,NaN,NaN,NaN,NaN,C:\Windows\system32\svchost.exe -k wsappx -p -s

In [68]:
pcreate_df = sysmon_df[~sysmon_df['CommandLine'].isnull()]

In [69]:
pcreate_df.head()

,@timestamp,channel,type,record_id,computer_name,version,provider_name,thread_id,provider_guid,opcode,event_id,task,process_id,level,TargetProcessId,SourceThreadId,SourceImage,GrantedAccess,CallTrace,UtcTime,SourceProcessGUID,TargetProcessGUID,SourceProcessId,TargetImage,SourceHostname,SourceIsIpv6,Protocol,ProcessGuid,SourceIp,SourcePortName,DestinationIp,DestinationPort,ProcessId,DestinationIsIpv6,SourcePort,Image,User,Initiated,DestinationHostname,PipeName,Company,Signed,Hashes,SignatureStatus,Product,ImageLoaded,FileVersion,Description,Signature,TargetObject,EventType,DestinationPortName,Details,Device,CommandLine,TerminalSessionId,CurrentDirectory,ParentProcessId,ParentCommandLine,IntegrityLevel,ParentProcessGuid,LogonGuid,LogonId,ParentImage,TargetFilename,CreationUtcTime,PreviousCreationUtcTime,Hash,sid,username,domain,type,0
241,2019-05-14T22:31:27.087Z,Microsoft-Windows-Sysmon/Operational,wineventlog,1447034.0,HR001.shire.com,3.0,Microsoft-Windows-Sysmon,3144.0,5770385f-c22a-43e0-bf4c-06f5698ffbd9,Info,10.0,Process accessed (rule: ProcessAccess),2440.0,Information,NaN,NaN,NaN,NaN,NaN,2019-05-14 22:31:27.319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03ba39f5-41bf-5cdb-0000-00104c568400,NaN,NaN,NaN,NaN,3336,NaN,NaN,C:\Windows\System32\taskhostw.exe,SHIRE\nmartha,NaN,NaN,NaN,Microsoft Corporation,NaN,"SHA1=121E6C3CF7AF4CA30DB43440F71C1510EC3EA261,MD5=F225F37717C8D714B56CF21C7C1E9C34,SHA256=E272D0E33299BCA60479683EAE0D9B5D880293C01B337D5759FEE812EA631A39,IMPHASH=9839C7FD9649496B162F72128209528A",NaN,Microsoft® Windows® Operating System,NaN,10.0.17763.1 (WinBuild.160101.0800),Host Process for Windows Tasks,NaN,NaN,NaN,NaN,NaN,NaN,taskhostw.exe KEYROAMING,1,C:\Windows\system32\,1048,C:\Windows\system32\svchost.exe -k netsvcs -p -s Schedule,Medium,03ba39f5-ea64-5ccb-0000-0010c91c0100,{03ba39f5-e67a-5cda-0000-0020c40c0c00},0xc0cc4,C:\Windows\System32\svchost.exe,NaN,NaN,NaN,NaN,S-1-5-18,SYSTEM,NT AUTHORITY,User,{5770385f-c22a-43e0-bf4c-06f5698ffbd9}
315,2019-05-14T22:31:27.324Z,Microsoft-Windows-Sysmon/Operational,wineventlog,1447094.0,HR001.shire.com,3.0,Microsoft-Windows-Sysmon,3144.0,5770385f-c22a-43e0-bf4c-06f5698ffbd9,Info,10.0,Process accessed (rule: ProcessAccess),2440.0,Information,NaN,NaN,NaN,NaN,NaN,2019-05-14 22:31:27.463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03ba39f5-41bf-5cdb-0000-0010785d8400,NaN,NaN,NaN,NaN,1128,NaN,NaN,C:\Windows\System32\backgroundTaskHost.exe,SHIRE\nmartha,NaN,NaN,NaN,Microsoft Corporation,NaN,"SHA1=339E4E69D2120B97CE34B9A8D3597FF8E0A73561,MD5=50D5FD1290D94D46ACCA0585311E74D5,SHA256=B8E176FE76A1454A00C4AF0F8BF8870650D9C33D3E333239A59445C5B35C9A37,IMPHASH=D2ACF1CBC4A6DB14A34C687B9362D66B",NaN,Microsoft® Windows® Operating System,NaN,10.0.17763.1 (WinBuild.160101.0800),Background Task Host,NaN,NaN,NaN,NaN,NaN,NaN,"""C:\Windows\system32\backgroundTaskHost.exe"" -ServerName:CortanaUI.AppXy7vb4pc2dr3kc93kfc509b1d0arkfb2x.mca",1,C:\Windows\SystemApps\Microsoft.Windows.Cortana_cw5n1h2txyewy\,748,C:\Windows\system32\svchost.exe -k DcomLaunch -p,AppContainer,03ba39f5-ea62-5ccb-0000-001096a70000,{03ba39f5-e67a-5cda-0000-0020c40c0c00},0xc0cc4,C:\Windows\System32\svchost.exe,NaN,NaN,NaN,NaN,S-1-5-18,SYSTEM,NT AUTHORITY,User,{5770385f-c22a-43e0-bf4c-06f5698ffbd9}
364,2019-05-14T22:31:27.357Z,Microsoft-Windows-Sysmon/Operational,wineventlog,1447143.0,HR001.shire.com,3.0,Microsoft-Windows-Sysmon,3144.0,5770385f-c22a-43e0-bf4c-06f5698ffbd9,Info,7.0,Image loaded (rule: ImageLoad),2440.0,Information,NaN,NaN,NaN,NaN,NaN,2019-05-14 22:31:27.680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03ba39f5-41bf-5cdb-0000-0010416a8400,NaN,NaN,NaN,NaN,2396,NaN,NaN,C:\Windows\System32\backgroundTaskHost.exe,SHIRE\nmartha,NaN,NaN,NaN,Microsoft Corporation,NaN,"SHA1=339E4E69D2120B97CE34B9A8D3597FF8E0A73561,MD5=50D5FD1290D94D46ACCA0585311E74D5,SHA256=B8E176FE76A1454A00C4AF0F8BF8870650D9C33D3E333239A59445C5B35C9A37,IMPHASH=D2ACF1CBC4A6DB14A34C687B9362D66B",NaN,Microsoft® Windows® Operating System,NaN,10.0.17763.1 (WinBuild.160101.0800),Background Task Host,NaN,NaN,Na

In [61]:
proc_dict = {}
proc_dict['root'] = []
# Find the root
for _, row in pcreate_df.iterrows():
    if any(pcreate_df['ProcessGuid'].isin([row['ParentProcessGuid']])):
        print(f'Child: {row["ProcessGuid"]} - Parent: {row["ParentProcessGuid"]}')
        continue
    else:
        proc_dict['root'].append(row['ProcessGuid'])
#         print(f'found root: {proc_dict["root"]}')
        proc_dict[row['ProcessGuid']] = Node(f'{row["ProcessGuid"]}', process=row["Image"], commandline=row["CommandLine"])
for _, row in pcreate_df.iterrows():
    t = proc_dict.get(row['ParentProcessGuid'], None)
    if t:
#         print(t)
#         print(row['ProcessGuid'])
        proc_dict[row['ProcessGuid']] = Node(f'{row["ProcessGuid"]}', process=row["Image"], commandline=row["CommandLine"], parent=proc_dict[row['ParentProcessGuid']])

Child: 03ba39f5-41bf-5cdb-0000-0010785d8400 - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: 03ba39f5-e68b-5cda-0000-0010d90b0e00 - Parent: nan
Child: 03ba39f5-e6e8-5cda-0000-0010e4ea1200 - Parent: nan
Child: 03ba39f5-41cf-5cdb-0000-001042648500 - Parent: nan
Child: 03ba39f5-41d2-5cdb-0000-0010c7978500 - Parent: nan
Child: 03ba39f5-41d4-5cdb-0000-001033c88500 - Parent: nan
Child: 03ba39f5-e6e7-5cda-0000-001092d71200 - Parent: nan
Child: 03ba39f5-e6e7-5cda-0000-001092d71200 - Parent: nan
Child: 03ba39f5-e6e8-5cda-0000-0010e4ea1200 - Parent: nan
Child: 03ba39f5-41e3-5cdb-0000-001096d08600 - Parent: nan
Child: 03ba39f5-41e3-5cdb-0000-001096d08600 - Parent: nan
Child: 03ba39f5-41e3-5cdb-0000-001096d08600 - Parent: nan
Child: 03ba39f5-e6b0-5cda-0000-00105c621000 - Parent: nan
Child: 03ba39f5-41f2-5cdb-0000-00106b3b8800 - Parent: nan
Child: 03ba39f5-41dc-5cdb-0000-0010ac688600 - Parent: nan
Child: 03ba39f5-41f4-5cdb-0000-0010cc5a8800 - Parent: nan
Child: 03ba39f5-41f6-5

Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan
Child: nan - Parent: nan


In [62]:
print(proc_dict['root'])
for pre, fill, node in RenderTree(proc_dict['a158f72c-623c-5de8-0000-0010b14b7400']):
    print("%s%s" % (pre, node.name))

[]


KeyError: 'a158f72c-623c-5de8-0000-0010b14b7400'

In [64]:
pcreate_df.head()

,@timestamp,channel,type,record_id,computer_name,version,provider_name,thread_id,provider_guid,opcode,event_id,task,process_id,level,TargetProcessId,SourceThreadId,SourceImage,GrantedAccess,CallTrace,UtcTime,SourceProcessGUID,TargetProcessGUID,SourceProcessId,TargetImage,SourceHostname,SourceIsIpv6,Protocol,ProcessGuid,SourceIp,SourcePortName,DestinationIp,DestinationPort,ProcessId,DestinationIsIpv6,SourcePort,Image,User,Initiated,DestinationHostname,PipeName,Company,Signed,Hashes,SignatureStatus,Product,ImageLoaded,FileVersion,Description,Signature,TargetObject,EventType,DestinationPortName,Details,Device,CommandLine,TerminalSessionId,CurrentDirectory,ParentProcessId,ParentCommandLine,IntegrityLevel,ParentProcessGuid,LogonGuid,LogonId,ParentImage,TargetFilename,CreationUtcTime,PreviousCreationUtcTime,Hash,sid,username,domain,type,0
321,2019-05-14T22:31:27.325Z,Microsoft-Windows-Sysmon/Operational,wineventlog,1447100.0,HR001.shire.com,5.0,Microsoft-Windows-Sysmon,3144.0,5770385f-c22a-43e0-bf4c-06f5698ffbd9,Info,1.0,Process Create (rule: ProcessCreate),2440.0,Information,NaN,NaN,NaN,NaN,NaN,2019-05-14 22:31:27.483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03ba39f5-41bf-5cdb-0000-0010785d8400,NaN,NaN,NaN,NaN,1128,NaN,NaN,C:\Windows\System32\backgroundTaskHost.exe,NaN,NaN,NaN,NaN,Microsoft Corporation,true,"SHA1=E5A45ED795E0BF6EF97EC6AFBCB5C4DC4329D845,MD5=A3C71F446CA92F389E9F27BC2C2CFBAF,SHA256=8FF185B57A235C6B05A52EE052822F3C8B893D802465585348DBECFB53D29106,IMPHASH=209056D2F9048DE4A76BAC3D98A22E98",Valid,Microsoft® Windows® Operating System,C:\Windows\System32\KernelBase.dll,10.0.17763.1 (WinBuild.160101.0800),Windows NT BASE API Client DLL,Microsoft Windows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S-1-5-18,SYSTEM,NT AUTHORITY,User,{5770385f-c22a-43e0-bf4c-06f5698ffbd9}
395,2019-05-14T22:31:27.492Z,Microsoft-Windows-Sysmon/Operational,wineventlog,1447174.0,HR001.shire.com,5.0,Microsoft-Windows-Sysmon,3144.0,5770385f-c22a-43e0-bf4c-06f5698ffbd9,Info,1.0,Process Create (rule: ProcessCreate),2440.0,Information,2396,6020,C:\Windows\system32\svchost.exe,0x100000,C:\Windows\SYSTEM32\ntdll.dll+a0724|C:\Windows\System32\RPCRT4.dll+4aeb4|c:\windows\system32\rpcss.dll+2e973|c:\windows\system32\rpcss.dll+34dc9|C:\Windows\System32\RPCRT4.dll+77803|C:\Windows\System32\RPCRT4.dll+1436c|C:\Windows\System32\RPCRT4.dll+1756a|C:\Windows\System32\RPCRT4.dll+54a38|C:\Windows\System32\RPCRT4.dll+304b0|C:\Windows\System32\RPCRT4.dll+2fe5b|C:\Windows\System32\RPCRT4.dll+221ff|C:\Windows\System32\RPCRT4.dll+2165a|C:\Windows\System32\RPCRT4.dll+20c21|C:\Windows\System32\RPCRT4.dll+20692|C:\Windows\System32\RPCRT4.dll+17465|C:\Windows\SYSTEM32\ntdll.dll+4f4c0|C:\Windows\SYSTEM32\ntdll.dll+50348|C:\Windows\System32\KERNEL32.DLL+17974|C:\Windows\SYSTEM32\ntdll.dll+6a271,2019-05-14 22:31:27.749,03ba39f5-ea63-5ccb-0000-001050e60000,03ba39f5-41bf-5cdb-0000-0010416a8400,780,C:\Windows\system32\backgroundTaskHost.exe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S-1-5-18,SYSTEM,NT AUTHORITY,User,{5770385f-c22a-43e0-bf4c-06f5698ffbd9}
444,2019-05-14T22:31:27.692Z,Microsoft-Windows-Sysmon/Operational,wineventlog,1447223.0,HR001.shire.com,5.0,Microsoft-Windows-Sysmon,3144.0,5770385f-c22a-43e0-bf4c-06f5698ffbd9,Info,1.0,Process Create (rule: ProcessCreate),2440.0,Information,1752,5396,C:\Windows\system32\svchost.exe,0x1000,C:\Windows\SYSTEM32\ntdll.dll+9fb54|C:\Windows\SYSTEM32\psmserviceexthost.dll+12ae4|C:\Windows\SYSTEM32\psmserviceexthost.dll+1295f|C:\Windows\SYSTEM32\psmserviceexthost.dll+1260e|C:\Windows\SYSTEM32\psmserviceexthost.dll+11e33|C:\Windows\SYSTEM32\ntdll.dll+64db0|C:\Windows\SYSTEM32\ntdll.dll+505c4|C:\Windows\System32\KERNEL32.DLL+17974|C:\Windows\SYSTEM32\ntdll.dll+6a271,2019-05-14 22:31:29.145,4209b313-e920-5cda-0000-001071a90000,4209b313-eb20-5cda-0000-00105cd60500,732,C:\Windows\System32\svcho

In [63]:
qgrid_widgets = qgrid.show_grid(pcreate_df, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200}, show_toolbar=True)
qgrid_widgets

AttributeError: 'DataFrame' object has no attribute 'map'

In [116]:
udo = Node("Udo")
marc = Node("Marc", parent=udo)
lian = Node("Lian", parent=marc)
jian = Node("Jian", parent=marc)
dan = Node("Dan", parent=udo)
jet = Node("Jet", parent=dan)
jan = Node("Jan", parent=dan)
joe = Node("Joe", parent=dan)
marc = Node("Marc", parent=udo)
marc = Node("Marc")

for pre, fill, node in RenderTree(marc):
    print("%s%s" % (pre, node.name))

Marc
